# BFS

In [2]:
import numpy as np
from collections import deque

class EightPuzzle:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.goal_state = np.array(
            [[1, 2, 3],
             [4, 5, 6],
             [7, 8, 0]]
        )
        self.max_row = 3
        self.max_col = 3
    
    def get_possible_moves(self, state):
        moves = []
        row, col = np.where(state==0)
        row, col = int(row[0]), int(col[0])
        directions = {
            "Up" : (-1, 0), # ("Up", (-1, 0))
            "Down" : (1, 0),
            "Left" : (0, -1),
            "Right" : (0, 1)
        }
        for move, (dr, dc) in directions.items():
            new_row, new_col = row+dr, col+dc
            if 0 <= new_row < self.max_row and 0 <= new_col < self.max_col:
                new_state = state.copy()
                new_state[row, col], new_state[new_row, new_col] = new_state[new_row, new_col], new_state[row, col]
                moves.append((new_state, move))
        return moves

    def bfs(self):
        frontier = deque([(self.initial_state, [])])
        explored = set()
        while frontier:
            current_state, path = frontier.popleft()
            if np.array_equal(current_state, self.goal_state):
                return path
            
            state_tuple = tuple(current_state.flatten())
            if state_tuple not in explored:
                explored.add(state_tuple)
                for next_state, move in self.get_possible_moves(current_state):
                    next_state_tuple = tuple(next_state.flatten())
                    if next_state_tuple not in explored:
                        frontier.append((next_state, path+[move]))
        return None

In [10]:
# initial_state = [
#     [1, 2, 3],
#     [4, 8, 5],
#     [7, 6, 0]
# ]

# initial_state = [
#     [1, 3, 8],
#     [4, 2, 5],
#     [7, 6, 0]
# ]

initial_state = [
    [1, 3, 8],
    [4, 2, 5],
    [7, 6, 0]
]

puzzle = EightPuzzle(np.array(initial_state))
solution = puzzle.bfs()

assert solution is not None

print(f"Initial state : \n{np.array(initial_state)}\n")

curr = (2, 2)
init_state = np.array(initial_state)
for idx, step in enumerate(solution):
    if step == 'Left':
        init_state[curr[0], curr[1]], init_state[curr[0], curr[1]-1] = init_state[curr[0], curr[1]-1], init_state[curr[0], curr[1]]
        curr = (curr[0], curr[1]-1)
    elif step == 'Right' :
        init_state[curr[0], curr[1]], init_state[curr[0], curr[1]+1] = init_state[curr[0], curr[1]+1], init_state[curr[0], curr[1]]
        curr = (curr[0], curr[1]+1)
    elif step == 'Up':
        init_state[curr[0], curr[1]], init_state[curr[0]-1, curr[1]] = init_state[curr[0]-1, curr[1]], init_state[curr[0], curr[1]]
        curr = (curr[0]-1, curr[1])
    elif step == 'Down':
        init_state[curr[0], curr[1]], init_state[curr[0]+1, curr[1]] = init_state[curr[0]+1, curr[1]], init_state[curr[0], curr[1]]
        curr = (curr[0]+1, curr[1])
    print(f"Step {idx+1} : {step}\n{init_state}\n")

Initial state : 
[[1 3 8]
 [4 2 5]
 [7 6 0]]

Step 1 : Up
[[1 3 8]
 [4 2 0]
 [7 6 5]]

Step 2 : Up
[[1 3 0]
 [4 2 8]
 [7 6 5]]

Step 3 : Left
[[1 0 3]
 [4 2 8]
 [7 6 5]]

Step 4 : Down
[[1 2 3]
 [4 0 8]
 [7 6 5]]

Step 5 : Down
[[1 2 3]
 [4 6 8]
 [7 0 5]]

Step 6 : Right
[[1 2 3]
 [4 6 8]
 [7 5 0]]

Step 7 : Up
[[1 2 3]
 [4 6 0]
 [7 5 8]]

Step 8 : Left
[[1 2 3]
 [4 0 6]
 [7 5 8]]

Step 9 : Down
[[1 2 3]
 [4 5 6]
 [7 0 8]]

Step 10 : Right
[[1 2 3]
 [4 5 6]
 [7 8 0]]



# UCS

In [11]:
import heapq

class Graph:
    def __init__(self):
        self.graph = {}
    
    def add_edge(self, node, neighbor, cost):
        if node not in self.graph:
            self.graph[node] = []
        self.graph[node].append((cost, neighbor))

    def uniform_cost_search(self, start, goal):
        priority_queue = []
        heapq.heappush(priority_queue, (0, start, []))

        visited = set()

        while priority_queue:
            cost, node, path = heapq.heappop(priority_queue)
            if node in visited:
                continue
            path = path + [node]
            visited.add(node)
            
            if node == goal:
                return path, cost
            
            for edge_cost, neighbor in self.graph.get(node, []):
                if neighbor not in visited:
                    heapq.heappush(priority_queue, (cost + edge_cost, neighbor, path))
        
        return None, float('inf')

In [12]:
graph = Graph()
graph.add_edge("A", "B", 1)
graph.add_edge("A", "C", 4)
graph.add_edge("B", "C", 2)
graph.add_edge("B", "D", 5)
graph.add_edge("C", "D", 1)
graph.add_edge("D", "E", 3)

# Bidirectional edges (UCS handles both)
graph.add_edge("B", "A", 1)
graph.add_edge("C", "A", 4)
graph.add_edge("C", "B", 2)
graph.add_edge("D", "B", 5)
graph.add_edge("D", "C", 1)
graph.add_edge("E", "D", 3)

start = "A"
end = "E"

path, cost = graph.uniform_cost_search(start, end)
print(f"Shortest path from {start} to {end} : {path} with cost {cost}")

Shortest path from A to E : ['A', 'B', 'C', 'D', 'E'] with cost 7
